In [1]:
import os 
import numpy as np 
import pandas as pd
#from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
#from imblearn.under_sampling import RandomUnderSampler
from sklearn.datasets import make_classification
import re

In [2]:

####################### 필요한 함수 준비 #########################
##내가 만든 내가 자주 사용하느 함수
#1) 자료 이름 바꾸기_ subject name 에 언더바나 이런게 있는 경우에 인식이 안되니까
def change_name (df):
  old = []
  new = []
  for i in df["subjectkey"]:
    old.append(i)
  
  for i in old:
    i = re.sub(r'_', '', i)
    new.append(i)

  df1 = df
  df1["subjectkey"] = new
  return df1

In [3]:
#============================ Change direc if needed ================================#
directory = '/storage/bigdata/ABCD/freesurfer/smri/fs_smri_brain_cropped'

In [4]:
def export_nda(directory):
    file_list = os.listdir(directory) 
    file_name = file_list[:]
    test_file = file_list[0]
    subjectkey = []
    print("how many participants were in this file:",len(file_name))
    ################ only nii,gz file #######################
    for part in file_name:
        if "nii" in part:
            pass
        else:
            file_name.remove(part)

    print("how many participants now in this file:",len(file_name))
    
    if "brain" in test_file:
        for part in file_name:
            a = re.search('NDA(.*?).brain',part).group(1)
            name = "NDA"+a
            subjectkey.append(name)
    else:
        for part in file_name:
            a = re.search('NDA(.*?).nii',part).group(1)
            name = "NDA"+a
            subjectkey.append(name)
            
    subjectkey = pd.DataFrame({"subjectkey": subjectkey})
    return subjectkey

In [5]:
mri_subkey = export_nda(directory)
mri_subkey

how many participants were in this file: 11374
how many participants now in this file: 11373


,subjectkey
0,NDARINVKHVXZ64U
1,NDARINV7W42V487
2,NDARINVKV3EVHUX
3,NDARINV4UCBM11Y
4,NDARINVN8WMY397
...,...
11368,NDARINVHTNVLRBR
11369,NDARINV32ZL36Y0
11370,NDARINVLDP9RT86
11371,NDARINVJF6WB6AE


# 기본적인 KSADS & CBCL 건강한지 안한지 지표 끌어오기

In [19]:
#load data
hc = pd.read_csv('./Healthy_ABCD.csv')
pheno = pd.read_csv('./ABCD_phenotype_total.csv')
ksads_C=pd.read_csv('./new_ksads_v2_child.csv')
ksads_P=pd.read_csv('./new_ksads_v2_parent.csv')
cbcl = pd.read_csv('./abcd_cbcls01_t.csv')

ksads_C = change_name(ksads_C)
ksads_C.drop('eventname',axis =1, inplace= True)

ksads_P = change_name(ksads_P)
ksads_P.drop('eventname',axis =1, inplace= True)

In [20]:
# MRI 지표에 맞추기
data = pd.merge(mri_subkey,hc, on = "subjectkey", how = "inner")
data

,subjectkey,HC_noCBCL,HC_noKSADS
0,NDARINVKHVXZ64U,0,0
1,NDARINV7W42V487,0,1
2,NDARINVKV3EVHUX,1,1
3,NDARINV4UCBM11Y,0,1
4,NDARINVN8WMY397,0,1
...,...,...,...
11130,NDARINVHTNVLRBR,0,1
11131,NDARINV32ZL36Y0,0,1
11132,NDARINVLDP9RT86,0,1
11133,NDARINVJF6WB6AE,0,0


In [22]:
#merge할 준비 
df = pd.merge(data,ksads_C, on = "subjectkey", how = "inner")
df = pd.merge(df,ksads_P, on = "subjectkey", how = "inner")
df = pd.merge(df,cbcl, on = "subjectkey", how = "inner")

# 부모랑 아이 둘다 1인 것만 추리기 어차피 N수가 지금 중요한 것이 아니니까

HC G2 MDD 이런식으로 이름짓기 

In [23]:
df_merged = df[:]

# 엄격한 기준을 적용 
- 해당 변수에 대해서 부모 설문 & 아동 설문 모두에서 1을 받아야 함. (Active & Present인 경우만) 
- HC : ksads(부모 + 아동) 에서 모두 0 / CBCL 에서도 정상군 N = 3027

- 이때 후보가 되는 열 중"SIP","SIA","SA","NSSI","ADHD","OPP","COND" 이 열들만 100명 이상 있었기 때문에 이들만 포함 _ 자살 관련해서는 주 교수님 논문 기준을 적용함


In [29]:
# columns 생성
df_merged["MDD"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "MDD" if (int(x.ksads_1_840_p + x.ksads_1_840_t) ==2)  else "SecHC" , axis = "columns") 

df_merged["Dysthymia"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "DYS" if (int(x.ksads_1_843_p + x.ksads_1_843_t) ==2)  else "SecHC" , axis = "columns") 

df_merged["GAD"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "GAD" if (int(x.ksads_10_869_p + x.ksads_10_869_t) ==2)  else "SecHC" , axis = "columns") 

df_merged["Other_SpAD"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "SpAD" if (int(x.ksads_10_913_p + x.ksads_10_913_t) ==2)  else "SecHC" , axis = "columns") 

df_merged["BD1_m"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "BD1_m" if (int(x.ksads_2_830_p + x.ksads_2_830_t) ==2)  else "SecHC" , axis = "columns") 

df_merged["BD1_d"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "BD1_d" if (int(x.ksads_2_831_p + x.ksads_2_831_t) ==2)  else "SecHC" , axis = "columns") 

df_merged["BD1_hym"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "BD1_hym" if (int(x.ksads_2_832_p + x.ksads_2_832_t) ==2)  else "SecHC" , axis = "columns") 

df_merged["BD2_hym"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "BD2_hym" if (int(x.ksads_2_835_p + x.ksads_2_835_t) ==2)  else "SecHC" , axis = "columns") 

df_merged["BD2_d"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "BD2_d" if (int(x.ksads_2_836_p + x.ksads_2_836_t) ==2)  else "SecHC" , axis = "columns") 

df_merged["SLEEP"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "SLEEP" if (int(x.ksads_22_969_p + x.ksads_22_969_t) ==2)  else "SecHC" , axis = "columns") 

df_merged["SAD"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "SAD" if (int(x.ksads_8_863_p + x.ksads_8_863_t) ==2)  else "SecHC" , axis = "columns") 

df_merged["SIP"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "SIP" if (int(x.SuicidalideationPassive_C + x.SuicidalideationPassive_P) ==2)  else "SecHC" , axis = "columns") 

df_merged["SIA"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "SIA" if (int(x.SuicidalideationActive_C + x.SuicidalideationActive_P) ==2)  else "SecHC" , axis = "columns") 

df_merged["SA"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "SA" if (int(x.Suicidalattempt_C + x.Suicidalattempt_P) ==2)  else "SecHC" , axis = "columns") 

df_merged["NSSI"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "NSSI" if (int(x.SelfInjuriousBehaviorwithoutsuicidalintent_C + x.SelfInjuriousBehaviorwithoutsuicidalintent_P) ==2)  else "SecHC" , axis = "columns") 



#naming_with CBCL
df_merged["ADHD"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "ADHD" if (int( x.ksads_14_853_p) == 1) and int(x.cbcl_scr_dsm5_adhd_t) >= 65 else "SecHC" , axis = "columns") 

df_merged["OPP"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "OPP" if (int( x.ksads_15_901_p) == 1) and int(x.cbcl_scr_dsm5_opposit_t) >= 65 else "SecHC" , axis = "columns") 

df_merged["COND"] = df_merged.apply(lambda x:"HC" if int(x.HC_noCBCL) == 0 and int(x.HC_noKSADS) == 0 else "COND" if (int( x.ksads_16_897_p) == 1) and int(x.cbcl_scr_dsm5_conduct_t) >= 65 else "SecHC" , axis = "columns") 


/home/connectome/mieuxmin/.conda/envs/PBS/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/connectome/mieuxmin/.conda/envs/PBS/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/connectome/mieuxmin/.conda/envs/PBS/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [32]:
df_merged_final = df_merged[["subjectkey","SIP","SIA","SA","NSSI","ADHD","OPP","COND"]]

In [33]:
df_merged_final

,subjectkey,SIP,SIA,SA,NSSI,ADHD,OPP,COND
0,NDARINVKHVXZ64U,HC,HC,HC,HC,HC,HC,HC
1,NDARINV7W42V487,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
2,NDARINVKV3EVHUX,SecHC,SecHC,SecHC,SecHC,ADHD,SecHC,SecHC
3,NDARINV4UCBM11Y,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
4,NDARINVN8WMY397,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
...,...,...,...,...,...,...,...,...
11130,NDARINVHTNVLRBR,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
11131,NDARINV32ZL36Y0,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
11132,NDARINVLDP9RT86,SIP,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
11133,NDARINVJF6WB6AE,HC,HC,HC,HC,HC,HC,HC


# phenotype과 합치기

In [36]:
pheno_usage = pheno[["subjectkey","age","sex","BMI","nihtbx_totalcomp_uncorrected",
                    "upps_total"]]
pheno_usage

,subjectkey,age,sex,BMI,nihtbx_totalcomp_uncorrected,upps_total
0,NDARINV003RTV85,131.0,2.0,20.480539,NaN,-0.370103
1,NDARINV005V6D2C,NaN,NaN,NaN,NaN,0.007040
2,NDARINV007W6H7B,126.0,1.0,18.234286,NaN,-0.998676
3,NDARINV00BD7VDC,112.0,1.0,16.329800,0.632147,-1.250105
4,NDARINV00CY2MDM,130.0,1.0,20.150208,-0.462016,-0.370103
...,...,...,...,...,...,...
11870,NDARINVZZNX6W2P,131.0,1.0,16.364477,0.632147,-2.255821
11871,NDARINVZZPKBDAC,113.0,2.0,18.253073,NaN,-5.147254
11872,NDARINVZZZ2ALR6,120.0,2.0,13.979693,NaN,1.012756
11873,NDARINVZZZNB0XC,108.0,2.0,18.446064,NaN,0.007040


In [48]:
df_merged_p = pd.merge(pheno_usage, df_merged_final, on = "subjectkey", how = "inner")
df_merged_p

,subjectkey,age,sex,BMI,nihtbx_totalcomp_uncorrected,upps_total,SIP,SIA,SA,NSSI,ADHD,OPP,COND
0,NDARINV003RTV85,131.0,2.0,20.480539,NaN,-0.370103,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
1,NDARINV007W6H7B,126.0,1.0,18.234286,NaN,-0.998676,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
2,NDARINV00BD7VDC,112.0,1.0,16.329800,0.632147,-1.250105,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
3,NDARINV00CY2MDM,130.0,1.0,20.150208,-0.462016,-0.370103,SecHC,SecHC,SecHC,SecHC,SecHC,OPP,SecHC
4,NDARINV00HEV6HB,124.0,1.0,15.173581,NaN,1.138470,HC,HC,HC,HC,HC,HC,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11130,NDARINVZZNX6W2P,131.0,1.0,16.364477,0.632147,-2.255821,HC,HC,HC,HC,HC,HC,HC
11131,NDARINVZZPKBDAC,113.0,2.0,18.253073,NaN,-5.147254,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
11132,NDARINVZZZ2ALR6,120.0,2.0,13.979693,NaN,1.012756,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
11133,NDARINVZZZNB0XC,108.0,2.0,18.446064,NaN,0.007040,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC


In [49]:
# 이름 바꾸기 
df_merged_p.rename(columns = {'subjectkey':'SubjectID'}, inplace = True)

In [50]:
#gender 변경
gen = []
for a in df_merged_p["sex"]:
    if a == 1:
        gen.append("M")
    elif a == 2:
        gen.append("W")
    else:
        gen.append(np.NaN)

df_merged_p["sex"] = gen
df_merged_p

,SubjectID,age,sex,BMI,nihtbx_totalcomp_uncorrected,upps_total,SIP,SIA,SA,NSSI,ADHD,OPP,COND
0,NDARINV003RTV85,131.0,W,20.480539,NaN,-0.370103,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
1,NDARINV007W6H7B,126.0,M,18.234286,NaN,-0.998676,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
2,NDARINV00BD7VDC,112.0,M,16.329800,0.632147,-1.250105,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
3,NDARINV00CY2MDM,130.0,M,20.150208,-0.462016,-0.370103,SecHC,SecHC,SecHC,SecHC,SecHC,OPP,SecHC
4,NDARINV00HEV6HB,124.0,M,15.173581,NaN,1.138470,HC,HC,HC,HC,HC,HC,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11130,NDARINVZZNX6W2P,131.0,M,16.364477,0.632147,-2.255821,HC,HC,HC,HC,HC,HC,HC
11131,NDARINVZZPKBDAC,113.0,W,18.253073,NaN,-5.147254,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
11132,NDARINVZZZ2ALR6,120.0,W,13.979693,NaN,1.012756,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC
11133,NDARINVZZZNB0XC,108.0,W,18.446064,NaN,0.007040,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC,SecHC


In [52]:
df_merged_p.to_csv("BT_ABCD_dataset.csv",index = False)